## Felles import for alle kodeblokker

In [ ]:
import sys
import calendar

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output

import ipywidgets as widgets
from IPython.display import display, clear_output

# Utvider søkesti til egne moduler
sys.path.append("../analysis")

from basedata import DataLoader
from monthlystats import MonthlyStats
from outlierdetector import OutlierDetector

# Initialisering av instanser
loader = DataLoader("../../data/processed")
ms = MonthlyStats("../../data/processed")
od = OutlierDetector()

## Tempratur forskjell by

In [5]:
# Hent ferdigberegnet temperaturspenn (range) for begge byer
df_oslo = ms.compute_all_months(
    "range(air_temperature P1D)",
    "oslo",
)
df_tromso = ms.compute_all_months(
    "range(air_temperature P1D)",
    "tromso",
)

# Legg til 'year', 'month' og 'temp_diff'
for df in (df_oslo, df_tromso):
    df["year_month"] = pd.to_datetime(df["year_month"])
    df["year"] = df["year_month"].dt.year
    df["month"] = df["year_month"].dt.month_name()
    df["temp_diff"] = df["mean"]  # Gi nytt navn for bruk i figuren

# Start Dash-app
app = Dash(__name__)


app.layout = html.Div(
    [
        html.H1("Temperature Range (Difference) per Month"),
        html.Label("City:"),
        dcc.Dropdown(
            id="city-dropdown",
            options=[
                {"label": "Oslo", "value": "Oslo"},
                {"label": "Tromsø", "value": "Tromsø"},
            ],
            value="Oslo",
            clearable=False,
        ),
        html.Label("Month:"),
        dcc.Dropdown(
            id="month-dropdown",
            options=[
                {"label": m, "value": m}
                for m in calendar.month_name[1:]
            ],
            value="January",
            clearable=False,
        ),
        html.Label("Start Year:"),
        dcc.Input(
            id="start-year",
            type="number",
            value=2000,
            min=2000,
            max=2023,
        ),
        html.Label("End Year:"),
        dcc.Input(
            id="end-year",
            type="number",
            value=2023,
            min=2000,
            max=2023,
        ),
        dcc.Graph(id="temperature-graph"),
    ],
)


@app.callback(
    Output("temperature-graph", "figure"),
    Input("city-dropdown", "value"),
    Input("month-dropdown", "value"),
    Input("start-year", "value"),
    Input("end-year", "value"),
)
def update_graph(
    city: str,
    month: str,
    start_year: int,
    end_year: int,
) -> go.Figure:
    """Oppdater bar-chart basert på brukerens valg."""
    source_df = df_oslo if city == "Oslo" else df_tromso

    df_filtered = source_df[
        (source_df["year"] >= start_year)
        & (source_df["year"] <= end_year)
        & (source_df["month"] == month)
    ]

    fig = px.bar(
        df_filtered,
        x="year",
        y="temp_diff",
        color="temp_diff",
        color_continuous_scale="YlOrRd",
        title=(
            f"Average Temperature Range for {month} – "
            f"{city} ({start_year} to {end_year})"
        ),
        labels={"temp_diff": "Temp. Range (°C)", "year": "Year"},
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True, port=8052)

## Sjekke data for bestemt dag basert på historikk

In [ ]:
# Slidere og dropdowns
dag_slider = widgets.IntSlider(
    value=17,
    min=1,
    max=31,
    description="Dag:",
)
maaned_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=12,
    description="Måned:",
)
by_dropdown = widgets.SelectMultiple(
    options=["oslo", "tromso"],
    value=["oslo"],
    description="By(er):",
)
element_dropdown = widgets.Dropdown(
    options=[
        "sum(precipitation_amount P1D)",
        "max(air_temperature P1D)",
        "mean(wind_speed P1D)",
    ],
    value="sum(precipitation_amount P1D)",
    description="Element:",
)
time_offset_dropdown = widgets.Dropdown(
    options=["PT0H", "PT6H", "PT12H", "PT18H"],
    value="PT0H",
    description="Time Offset:",
)
knapp = widgets.Button(description="Vis statistikk")
output = widgets.Output()


def vis_statistikk(b) -> None:
    """Vis gjennomsnitt og outlier-info for valgt dag, måned og by(er)."""
    with output:
        clear_output()
        dag = dag_slider.value
        maaned = maaned_slider.value
        valgte_byer = by_dropdown.value
        element_id = element_dropdown.value
        time_offset = time_offset_dropdown.value
        visningsnavn = {"oslo": "Oslo", "tromso": "Tromsø"}
        stats_data: dict[str, float] = {}

        # Hent gjennomsnitt per by
        for by in valgte_byer:
            try:
                df = loader._load_city(by)
                df_filtered = df[
                    (df["elementId"] == element_id)
                    & (df["timeOffset"] == time_offset)
                ].copy()
                df_filtered["referenceTime"] = pd.to_datetime(
                    df_filtered["referenceTime"], utc=True
                )
                df_filtered = df_filtered[
                    (df_filtered["referenceTime"].dt.month == maaned)
                    & (df_filtered["referenceTime"].dt.day == dag)
                ]
                df_filtered["value"] = pd.to_numeric(
                    df_filtered["value"], errors="coerce"
                )

                if not df_filtered.empty:
                    stats_data[visningsnavn[by]] = df_filtered["value"].mean()
            except Exception as e:
                print(f"Feil ved behandling av data for {by}: {e}")

        if not stats_data:
            print("Ingen data tilgjengelig for valgt dag/by.")
            return

        # Plott gjennomsnitt
        plt.figure(figsize=(10, 5))
        plt.bar(stats_data.keys(), stats_data.values(), edgecolor="black")
        plt.title(f"Gjennomsnittlig {element_id} for {dag:02d}.{maaned:02d}")
        plt.ylabel("Verdi")
        plt.grid(axis="y")
        plt.tight_layout()
        plt.show()

        # Skriv ut gjennomsnittsverdier
        print("\nGjennomsnittsverdier:")
        for navn, verdi in stats_data.items():
            print(f"{navn}: {verdi:.2f}")

        # Outlier-informasjon
        print("\nOutlier-informasjon:")
        for by_kode in valgte_byer:
            try:
                df = loader._load_city(by_kode)
                df_filtered = df[
                    (df["elementId"] == element_id)
                    & (df["timeOffset"] == time_offset)
                ].copy()
                df_filtered["value"] = pd.to_numeric(
                    df_filtered["value"], errors="coerce"
                )
                mask = od.detect_iqr(df_filtered["value"])
                count = int(mask.sum())

                navn = visningsnavn[by_kode]
                if count > 0:
                    print(f"{navn}: {count} outliers funnet")
                else:
                    print(f"{navn}: Ingen outliers funnet")
            except Exception as e:
                print(f"Kunne ikke analysere outliers for {navn}: {e}")


knapp.on_click(vis_statistikk)
display(
    maaned_slider,
    dag_slider,
    by_dropdown,
    element_dropdown,
    time_offset_dropdown,
    knapp,
    output,
)

IntSlider(value=5, description='Måned:', max=12, min=1)

IntSlider(value=17, description='Dag:', max=31, min=1)

SelectMultiple(description='By(er):', index=(0,), options=('oslo', 'tromso'), value=('oslo',))

Dropdown(description='Element:', options=('sum(precipitation_amount P1D)', 'max(air_temperature P1D)', 'mean(w…

Dropdown(description='Time Offset:', options=('PT0H', 'PT6H', 'PT12H', 'PT18H'), value='PT0H')

Button(description='Vis statistikk', style=ButtonStyle())

Output()